In [1]:
#import packages
import os
import h5py
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
data_dir='Outputs/'
from itertools import islice
from scipy import interpolate
import pyproj
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


# make sure we're dealing with the most recent version of any code we're using
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
#create functions

#by Suzanne Dickinson
def moving_average(iterable, n):
    # create an iterable object from input argument
    it = iter(iterable)
    piece = list(islice(it, n))
    while piece:
        yield np.mean(piece), np.median(piece), np.std(piece)
        # yield gives back a generator, which needs to be iterated upon to get the info out.
        piece = list(islice(it, n))
        
#from Fernando Paolo & Johan Nilsson's utilities
def transform_coord(proj1, proj2, x, y):
    """Transform coordinates from proj1 to proj2 (EPSG num)."""
    
    # Set full EPSG projection strings
    proj1 = pyproj.Proj("+init=EPSG:"+proj1)
    proj2 = pyproj.Proj("+init=EPSG:"+proj2)
    
    # Convert coordinates
    return pyproj.transform(proj1, proj2, x, y)

In [16]:
%matplotlib widget

#identify files
ATL03_file=glob('/home/jovyan/crackup/Outputs_New/Edgeworth*ATL03*.h5')
ATL06_file=glob('/home/jovyan/crackup/Outputs_New/Edgeworth*ATL06*.h5')
beamNum = [1,3,5]

#read data
f = h5py.File(ATL03_file[0], 'r')  # keep it open
beam = [k for k in f.keys() if k.startswith('gt')]
f6 = h5py.File(ATL06_file[0], 'r')
beam6 = [k for k in f6.keys() if k.startswith('gt')]


#set up dictionary
lookfor = ['delta_time','h_li','h_li_sigma','latitude','longitude','segment_id','sigma_geo_h']

#create base figures
fig1, ax1 = plt.subplots(nrows=2, ncols=1, sharex=True, sharey=True, figsize=(8,8))
fig2, ax2 = plt.subplots(nrows=2, ncols=1, sharex=True, sharey=True, figsize=(8,8))

#pull data and fill-in structures
data = {}
data6 = {}
window = 20 #specify moving window size
for ii,p in enumerate(beamNum):
    print('beam',beam[p])
    data[beam[p]] = {}
    data[beam[p]]['heights'] = {}

    for key,val in f[beam[p]]['heights'].items():
        data[beam[p]]['heights'][key] = val[:]

    #-- 0=Land; 1=Ocean; 2=SeaIce; 3=LandIce; 4=InlandWater
    conf = data[beam[p]]['heights']['signal_conf_ph'][:,3]
    lat_ph = np.array(data[beam[p]]['heights']['lat_ph'][conf>=2])
    lon_ph = np.array(data[beam[p]]['heights']['lon_ph'][conf>=2])
    h_ph   = np.array(data[beam[p]]['heights']['h_ph'][conf>=2])
    t_ph   = np.array(data[beam[p]]['heights']['delta_time'][conf>=2])
    
    havg = np.array([])
    hmed = np.array([])
    hstd= np.array([])
    for stats in moving_average(h_ph,window):
        havg = np.append(havg,stats[0])
        hmed = np.append(hmed,stats[1])
        hstd = np.append(hstd,stats[2])

    lat_avg = np.array([])
    for stats in moving_average(lat_ph,window):
        lat_avg = np.append(lat_avg,stats[0])
    lon_avg = np.array([])
    for stats in moving_average(lon_ph,window):
        lon_avg = np.append(lon_avg,stats[0])
    x_avg, y_avg = transform_coord('4326', '3031', lon_avg, lat_avg)

    data6[beam6[p]] = {}
    data6[beam6[p]]['land_ice_segments'] = {}
    
    for key,val in f6[beam6[p]]['land_ice_segments'].items():
        if key in lookfor:
            data6[beam6[p]]['land_ice_segments'][key] = val[:]
    h_li = data6[beam6[p]]['land_ice_segments']['h_li']
    h_li[h_li>3e38]= np.nan
    lat_li = data6[beam6[p]]['land_ice_segments']['latitude']
    lon_li = data6[beam6[p]]['land_ice_segments']['longitude']
    print(len(lon_li))
    x_li, y_li = transform_coord('4326', '3031', lon_li, lat_li)

    # print(find_nearest(pointlat,photlat[200]))
    elev_diff = np.zeros_like(y_avg)
    for i,reflat in enumerate(y_avg):
        dist_array = np.sqrt((x_li-x_avg[i])**2 + (y_li-reflat)**2)
        idx = np.where(dist_array == dist_array.min())
    #     idx = find_nearest(pointlat,reflat)
    #     if np.abs(pointlat[idx]-photlat[i]) <= 20:
        if dist_array[idx] <= 20:
            elev_diff[i] = h_li[idx] - havg[i]
        else:
            elev_diff[i] = np.NaN
    #         photlat[i] = 0
    
    if ii<2:
        ax1[ii].plot(lat_ph,h_ph,'.',color=(0.3,0.3,0.3),markersize=0.7,label='ATL03')
        ax1[ii].plot(lat_avg,havg,'r',linewidth=0.8,label='moving average')
        ax1[ii].plot(lat_avg,hmed,'g',linewidth=0.6,label='moving median')
        ax1[ii].plot(lat_avg,havg+hstd,'c--',linewidth=0.3,label='moving stdev')
        ax1[ii].plot(lat_avg,havg-hstd,'c--',linewidth=0.3)
        ax1[ii].plot(lat_li,h_li,'k.',label='ATL06')
        ax1[ii].set_title(beam[p])
#         ax1[ii].set_ylim(0,80)
#         ax1[ii].set_xlim(-64.38,-64.37)

        ax2[ii].plot(lat_avg,elev_diff,'.',color=(0.3,0.3,0.3),markersize=0.7,label='ATL06 minus ATL03')
        ax2[ii].set_title(beam[p])
#         ax2[ii].set_ylim(-10,60)
#         ax2[ii].set_xlim(-64.38,-64.37)

lgd = ax1[0].legend(loc=0,frameon=True)
lgd.get_frame().set_alpha(1.0)
for line in lgd.get_lines():
    line.set_linewidth(6)

lgd = ax2[0].legend(loc=2,frameon=True)
lgd.get_frame().set_alpha(1.0)
for line in lgd.get_lines():
    line.set_linewidth(6)


plt.show()
f.close()
f6.close()


FigureCanvasNbAgg()

FigureCanvasNbAgg()

beam gt1r
698
beam gt2r
700
beam gt3r
742


In [22]:
#print the statistics out
print('Mean ATL06-ATL03 elevations = ',np.around(np.nanmean(elev_diff),2),' m')
print('Median ATL06-ATL03 elevations = ',np.around(np.nanmedian(elev_diff),2),' m')
print('St. dev. in ATL06-ATL03 elevations = ',np.around(np.nanstd(elev_diff),2),' m')

Mean ATL06-ATL03 elevations =  0.12  m
Median ATL06-ATL03 elevations =  0.08  m
St. dev. in ATL06-ATL03 elevations =  1.32  m
